In [1]:
from QuantLib import *
from collections import namedtuple
import math
from pandas import DataFrame

In [2]:
# This is for compatibility with QuantLib < 1.15
try:
    BlackCalibrationHelper
except:
    BlackCalibrationHelper = CalibrationHelper

In [3]:
today = Date(15, February, 2002);
settlement= Date(19, February, 2002);
Settings.instance().evaluationDate = today;
term_structure = YieldTermStructureHandle(FlatForward(settlement,0.04875825,Actual365Fixed()))
index = Euribor1Y(term_structure)

In [4]:
CalibrationData = namedtuple("CalibrationData", "start, length, volatility")
data = [CalibrationData(1, 5, 0.1148),CalibrationData(2, 4, 0.1108),CalibrationData(3, 3, 0.1070),CalibrationData(4, 2, 0.1021),CalibrationData(5, 1, 0.1000 )]

In [5]:
def create_swaption_helpers(data, index, term_structure, engine):
    swaptions = []
    fixed_leg_tenor = Period(1, Years)
    fixed_leg_daycounter = Actual360()
    floating_leg_daycounter = Actual360()
    for d in data:
        vol_handle = QuoteHandle(SimpleQuote(d.volatility))
        helper = SwaptionHelper(Period(d.start, Years),Period(d.length, Years),vol_handle,index,fixed_leg_tenor,fixed_leg_daycounter,floating_leg_daycounter,term_structure)
        helper.setPricingEngine(engine)
        swaptions.append(helper)
    return swaptions

In [6]:
def calibration_report(swaptions, data):
    columns = ["Model Price", "Market Price", "Implied Vol", "Market Vol","Rel Error Price", "Rel Error Vols"]
    report_data = []
    cum_err = 0.0
    cum_err2 = 0.0
    for i, s in enumerate(swaptions):
        model_price = s.modelValue()
        market_vol = data[i].volatility
        black_price = s.blackPrice(market_vol)
        rel_error = model_price/black_price - 1.0
        implied_vol = s.impliedVolatility(model_price,1e-5, 50, 0.0, 0.50)
        rel_error2 = implied_vol/market_vol-1.0
        cum_err += rel_error*rel_error
        cum_err2 += rel_error2*rel_error2
        report_data.append((model_price, black_price, implied_vol, market_vol, rel_error, rel_error2))
    print("Cumulative Error Price: %7.5f" % math.sqrt(cum_err))
    print("Cumulative Error Vols : %7.5f" % math.sqrt(cum_err2))
    return DataFrame(report_data,columns= columns, index=['']*len(report_data))

In [7]:
    model = HullWhite(term_structure)
    engine = JamshidianSwaptionEngine(model)
    swaptions = create_swaption_helpers(data, index, term_structure, engine)
    optimization_method = LevenbergMarquardt(1.0e-8,1.0e-8,1.0e-8)
    end_criteria = EndCriteria(10000, 100, 1e-6, 1e-8, 1e-8)
    model.calibrate(swaptions, optimization_method, end_criteria)    
    a, sigma = model.params()
    print("a = %6.5f, sigma = %6.5f" % (a, sigma))
    calibration_report(swaptions, data)

a = 0.04642, sigma = 0.00580
Cumulative Error Price: 0.11583
Cumulative Error Vols : 0.11614


Model Price  Market Price  Implied Vol  Market Vol  Rel Error Price  \
     0.008775      0.009485     0.106198      0.1148        -0.074854   
     0.009669      0.010078     0.106292      0.1108        -0.040610   
     0.008663      0.008716     0.106343      0.1070        -0.006138   
     0.006490      0.006226     0.106442      0.1021         0.042367   
     0.003542      0.003323     0.106612      0.1000         0.065817   

  Rel Error Vols  
       -0.074928  
       -0.040688  
       -0.006138  
        0.042525  
        0.066122

In [8]:
    constrained_model = HullWhite(term_structure, 0.05, 0.001);
    engine = JamshidianSwaptionEngine(constrained_model)
    swaptions = create_swaption_helpers(data, index, term_structure, engine)
    optimization_method = LevenbergMarquardt(1.0e-8,1.0e-8,1.0e-8)
    end_criteria = EndCriteria(10000, 100, 1e-6, 1e-8, 1e-8)
    constrained_model.calibrate(swaptions, optimization_method,end_criteria, NoConstraint(),[], [True, False])
    a, sigma = constrained_model.params()
    print("a = %6.5f, sigma = %6.5f" % (a, sigma))
    calibration_report(swaptions, data)

a = 0.05000, sigma = 0.00586
Cumulative Error Price: 0.11584
Cumulative Error Vols : 0.11615


Model Price  Market Price  Implied Vol  Market Vol  Rel Error Price  \
     0.008776      0.009485     0.106212      0.1148        -0.074738   
     0.009668      0.010078     0.106284      0.1108        -0.040682   
     0.008662      0.008716     0.106330      0.1070        -0.006261   
     0.006490      0.006226     0.106436      0.1021         0.042311   
     0.003542      0.003323     0.106625      0.1000         0.065946   

  Rel Error Vols  
       -0.074812  
       -0.040761  
       -0.006261  
        0.042469  
        0.066252

In [9]:
    model = BlackKarasinski(term_structure);
    engine = TreeSwaptionEngine(model, 100)
    swaptions = create_swaption_helpers(data, index, term_structure, engine)
    optimization_method = LevenbergMarquardt(1.0e-8,1.0e-8,1.0e-8)
    end_criteria = EndCriteria(10000, 100, 1e-6, 1e-8, 1e-8)
    model.calibrate(swaptions, optimization_method, end_criteria)
    a, sigma = model.params()
    print("a = %6.5f, sigma = %6.5f" % (a, sigma))
    calibration_report(swaptions, data)

a = 0.03902, sigma = 0.11695
Cumulative Error Price: 0.12132
Cumulative Error Vols : 0.12163


Model Price  Market Price  Implied Vol  Market Vol  Rel Error Price  \
     0.008717      0.009485     0.105497      0.1148        -0.080954   
     0.009670      0.010078     0.106309      0.1108        -0.040453   
     0.008679      0.008716     0.106540      0.1070        -0.004297   
     0.006503      0.006226     0.106656      0.1021         0.044457   
     0.003547      0.003323     0.106765      0.1000         0.067333   

  Rel Error Vols  
       -0.081033  
       -0.040531  
       -0.004297  
        0.044623  
        0.067646

In [10]:
    model = G2(term_structure);
    engine = TreeSwaptionEngine(model, 25)
    # engine = ql.G2SwaptionEngine(model, 10, 400)
    # engine = ql.FdG2SwaptionEngine(model)
    swaptions = create_swaption_helpers(data, index, term_structure, engine)
    optimization_method = LevenbergMarquardt(1.0e-8,1.0e-8,1.0e-8)
    end_criteria = EndCriteria(1000, 100, 1e-6, 1e-8, 1e-8)
    model.calibrate(swaptions, optimization_method, end_criteria)
    a, sigma, b, eta, rho = model.params()
    print("a = %6.5f, sigma = %6.5f, b = %6.5f, eta = %6.5f, rho = %6.5f " % (a, sigma, b, eta, rho))
    calibration_report(swaptions, data)

a = 0.03927, sigma = 0.00475, b = 0.04545, eta = 0.00303, rho = 0.02206 
Cumulative Error Price: 0.12207
Cumulative Error Vols : 0.12238


Model Price  Market Price  Implied Vol  Market Vol  Rel Error Price  \
     0.008707      0.009485     0.105372      0.1148        -0.082041   
     0.009672      0.010078     0.106328      0.1108        -0.040281   
     0.008676      0.008716     0.106508      0.1070        -0.004602   
     0.006502      0.006226     0.106639      0.1021         0.044291   
     0.003548      0.003323     0.106788      0.1000         0.067562   

  Rel Error Vols  
       -0.082122  
       -0.040359  
       -0.004602  
        0.044457  
        0.067877